<a href="https://colab.research.google.com/github/Capstone-Team-CH2-PS379/Machine-Learning/blob/main/model-experiments/Speech_Recognition_Model_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Data

In [14]:
!pip install kaggle

In [ ]:
# Library
import os
import pathlib
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

In [ ]:
# Set Kaggle API Key as environment variable
os.environ['KAGGLE_USERNAME'] = os.environ['KAGGLE_KEY'] = 

In [ ]:
# Download dataset
!kaggle datasets download -d mozillaorg/common-voice

100% 12.0G/12.0G [10:45<00:00, 23.0MB/s]
100% 12.0G/12.0G [10:45<00:00, 20.0MB/s]


In [ ]:
# Extract zipfile to folder
import zipfile

with zipfile.ZipFile('common-voice.zip', 'r') as zip_ref:
    zip_ref.extractall('common-voice')

In [ ]:
print(os.listdir('/content/common-voice'))

['cv-valid-train', 'cv-other-test.csv', 'cv-invalid', 'cv-other-train', 'cv-other-dev', 'cv-other-test', 'cv-valid-dev', 'cv-valid-test', 'cv-invalid.csv', 'cv-other-train.csv', 'cv-other-dev.csv', 'cv-valid-train.csv', 'LICENSE.txt', 'cv-valid-dev.csv', 'cv-valid-test.csv', 'README.txt']


In [15]:
# Read the CSV file
file_path_train = '/content/common-voice/cv-valid-train.csv'
data_train = pd.read_csv(file_path_train)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195776 entries, 0 to 195775
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   filename    195776 non-null  object 
 1   text        195776 non-null  object 
 2   up_votes    195776 non-null  int64  
 3   down_votes  195776 non-null  int64  
 4   age         73768 non-null   object 
 5   gender      74059 non-null   object 
 6   accent      64711 non-null   object 
 7   duration    0 non-null       float64
dtypes: float64(1), int64(2), object(5)
memory usage: 11.9+ MB


In [16]:
# Set the option to display maximum column width
pd.set_option('display.max_colwidth', None)
data_train.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-000000.mp3,learn to recognize omens and follow them the old king had said,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-000001.mp3,everything in the universe evolved he said,1,0,NaN,NaN,NaN,NaN
2,cv-valid-train/sample-000002.mp3,you came so that you could learn about your dreams said the old woman,1,0,NaN,NaN,NaN,NaN
3,cv-valid-train/sample-000003.mp3,so now i fear nothing because it was those omens that brought you to me,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-000004.mp3,if you start your emails with greetings let me be the first to welcome you to earth,3,2,NaN,NaN,NaN,NaN


In [17]:
file_path_val = '/content/common-voice/cv-valid-dev.csv'
data_val = pd.read_csv(file_path_val)
data_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   filename    4076 non-null   object 
 1   text        4076 non-null   object 
 2   up_votes    4076 non-null   int64  
 3   down_votes  4076 non-null   int64  
 4   age         1528 non-null   object 
 5   gender      1540 non-null   object 
 6   accent      1350 non-null   object 
 7   duration    0 non-null      float64
dtypes: float64(1), int64(2), object(5)
memory usage: 254.9+ KB


In [18]:
data_val.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the stranger,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they see one,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage entered and greeted the englishman,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be destroyed,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could hear him,1,0,fourties,female,england,NaN


# Tokenization

In [19]:
# Tokenisasi kolom 'text' menggunakan library nltk atau library NLP lainnya
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # Download data yang dibutuhkan oleh nltk
data_train['tokenized_text'] = data_train['text'].apply(lambda x: word_tokenize(x.lower()))

# Periksa hasil tokenisasi
print(data_train[['text', 'tokenized_text']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


                                                                                  text  \
0                       learn to recognize omens and follow them the old king had said   
1                                           everything in the universe evolved he said   
2                you came so that you could learn about your dreams said the old woman   
3              so now i fear nothing because it was those omens that brought you to me   
4  if you start your emails with greetings let me be the first to welcome you to earth   

                                                                                          tokenized_text  
0                            [learn, to, recognize, omens, and, follow, them, the, old, king, had, said]  
1                                                     [everything, in, the, universe, evolved, he, said]  
2                   [you, came, so, that, you, could, learn, about, your, dreams, said, the, old, woman]  
3                [so, now, i, f

In [20]:
# Tokenisasi kolom 'text' pada data validasi
data_val['tokenized_text'] = data_val['text'].apply(lambda x: word_tokenize(x.lower()))

# Periksa hasil tokenisasi
print(data_val[['text', 'tokenized_text']].head())

                                                                            text  \
0                        be careful with your prognostications said the stranger   
1                            then why should they be surprised when they see one   
2  a young arab also loaded down with baggage entered and greeted the englishman   
3                           i thought that everything i owned would be destroyed   
4                           he moved about invisible but everyone could hear him   

                                                                                tokenized_text  
0                             [be, careful, with, your, prognostications, said, the, stranger]  
1                               [then, why, should, they, be, surprised, when, they, see, one]  
2  [a, young, arab, also, loaded, down, with, baggage, entered, and, greeted, the, englishman]  
3                               [i, thought, that, everything, i, owned, would, be, destroyed]  
4         

In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Membuat Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_train['tokenized_text'])

sample_size_train = 1000
sample_size_val = 100

# Konversi token menjadi indeks numerik
X_train_sequence = tokenizer.texts_to_sequences(data_train['tokenized_text'][:sample_size_train])
X_val_sequence = tokenizer.texts_to_sequences(data_val['tokenized_text'][:sample_size_val])

print(X_train_sequence)
print(X_val_sequence)


[[332, 2, 1289, 187, 4, 1078, 57, 1, 69, 243, 15, 18], [126, 12, 1, 1522, 1543, 7, 18], [10, 98, 46, 11, 10, 71, 332, 32, 47, 383, 18, 1, 69, 206], [46, 88, 3, 369, 119, 91, 8, 9, 117, 187, 11, 308, 10, 2, 31], [60, 10, 348, 47, 966, 22, 1005, 144, 31, 30, 1, 108, 2, 919, 10, 2, 208], [5, 297, 307, 73, 2, 384, 24, 7, 139, 76, 389, 32, 17, 134], [111, 222, 4, 63, 2562, 6, 747, 125, 4, 1303, 748, 4, 2563, 1, 329], [3, 174, 5, 637, 295, 574, 26, 582], [267, 4355, 146, 33, 1, 4356], [58, 7, 1079, 12, 188, 247, 3, 62, 805, 28, 7, 116, 18], [10, 135, 40, 144, 64, 143, 173, 419, 141, 46, 260, 7, 301], [60, 10, 27, 2, 290, 275, 1, 226, 20, 109, 55, 290], [26, 575, 317, 44, 2, 31, 1, 1087, 6, 1, 511, 402, 4, 742, 487, 403], [24, 7, 15, 107, 5, 1502, 4, 53, 16, 79, 2, 562, 44, 33, 63, 2203], [38, 21, 157, 2, 3052, 1, 4357, 5, 4358, 220, 12, 1, 1346], [3, 1690, 333, 621, 25, 8], [7, 9, 26, 1494], [24, 7, 249, 2, 197, 5, 1248], [42, 90, 396, 41, 47, 4554], [55, 21, 95, 5101, 6, 74, 2873, 4, 106, 5

## Feature Engineering

In [131]:
import librosa

# Contoh ekstraksi fitur MFCC dari file audio
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs

# Gunakan fungsi di atas untuk menghasilkan fitur dari file audio
file_path_example = '/content/common-voice/cv-valid-train/cv-valid-train/sample-000000.mp3'
mfcc_example = extract_mfcc(file_path_example)

# Periksa bentuk fitur yang dihasilkan
print("MFCC Shape:", mfcc_example.shape)

MFCC Shape: (13, 385)


In [107]:
audio_path_train = '/content/common-voice/cv-valid-train/'
audio_path_val = '/content/common-voice/cv-valid-dev/'

sample_size_train = 1000
sample_size_val = 100

# Menangani dimensi yang berbeda
features_train = [extract_mfcc(audio_path_train + file) for file in data_train['filename'][:sample_size_train]]
features_val = [extract_mfcc(audio_path_val + file) for file in data_val['filename'][:sample_size_val]]

# Menyesuaikan dimensi agar konsisten
max_dim_train = max(feature.shape[1] for feature in features_train)
#max_dim_val = max(feature.shape[1] for feature in features_val)

features_train_processed = [np.pad(feature, ((0, 0), (0, max_dim_train - feature.shape[1]))) for feature in features_train]
features_val_processed = [np.pad(feature, ((0, 0), (0,  max_dim_train - feature.shape[1]))) for feature in features_val]

X_train = np.array(features_train_processed)
X_val = np.array(features_val_processed)

In [104]:
print(max_dim_train)

1110


In [119]:
print(X_train.shape)
print(X_val.shape)

(1000, 13, 1110)
(100, 13, 1110)


In [120]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Konversi setiap sampel menjadi one-hot encoding
y_train = pad_sequences(X_train_sequence, padding='post')
y_val = pad_sequences(X_val_sequence, padding='post')

# Konversi ke one-hot encoding menggunakan to_categorical
y_train = np.array([to_categorical(seq, num_classes=len(tokenizer.word_index) + 1) for seq in y_train])
y_val = np.array([to_categorical(seq, num_classes=len(tokenizer.word_index) + 1) for seq in y_val])

In [121]:
model = tf.keras.models.Sequential()

# Lapisan Konvolusi dan Pooling
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(13, max_dim_train, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Lapisan Flatten dan Dense
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))

# Lapisan Output
num_classes = len(tokenizer.word_index) + 1
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 11, 1108, 32)      320       
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 5, 554, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_28 (Conv2D)          (None, 3, 552, 64)        18496     
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 1, 276, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_18 (Flatten)        (None, 17664)             0         
                                                                 
 dense_27 (Dense)            (None, 128)             

In [114]:
print(y_train.shape)
print(y_val.shape)

(1000, 27, 7844)
(100, 27, 7844)


In [115]:
# Ambil distribusi kelas pada langkah waktu terakhir
y_train_last_step = y_train[:, -1, :]
y_val_last_step = y_val[:, -1, :]

# Pastikan bentuknya sesuai
print("Shape y_train_last_step:", y_train_last_step.shape)
print("Shape y_val_last_step:", y_val_last_step.shape)

Shape y_train_last_step: (1000, 7844)
Shape y_val_last_step: (100, 7844)


In [122]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [123]:
model.fit(X_train,
          y_train_last_step,
          epochs=10,
          batch_size=32,
          validation_data=(X_val, y_val_last_step))

Epoch 1/10
32/32 [==============================] - 3s 30ms/step - loss: 0.8837 - accuracy: 0.9660 - val_loss: 1.3536 - val_accuracy: 0.9900
Epoch 2/10
32/32 [==============================] - 1s 16ms/step - loss: 0.0418 - accuracy: 0.9960 - val_loss: 0.5170 - val_accuracy: 0.9900
Epoch 3/10
32/32 [==============================] - 0s 15ms/step - loss: 0.0456 - accuracy: 0.9980 - val_loss: 0.4162 - val_accuracy: 0.9900
Epoch 4/10
32/32 [==============================] - 0s 14ms/step - loss: 0.0280 - accuracy: 0.9950 - val_loss: 0.4609 - val_accuracy: 0.9900
Epoch 5/10
32/32 [==============================] - 0s 13ms/step - loss: 9.9882e-05 - accuracy: 1.0000 - val_loss: 0.6889 - val_accuracy: 0.9900
Epoch 6/10
32/32 [==============================] - 0s 15ms/step - loss: 2.7207e-06 - accuracy: 1.0000 - val_loss: 0.6892 - val_accuracy: 0.9900
Epoch 7/10
32/32 [==============================] - 0s 14ms/step - loss: 2.6864e-05 - accuracy: 1.0000 - val_loss: 0.6928 - val_accuracy: 0.9900
E

In [124]:
# Simpan model
model.save('/content/model_v1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Model Testing

In [132]:
file_path_test = '/content/common-voice/cv-valid-train/cv-valid-train/sample-000000.mp3'
mfcc_test = extract_mfcc(file_path_test)

In [133]:
mfcc_test_processed = np.pad(mfcc_test, ((0, 0), (0, max_dim_train - mfcc_test.shape[1])))

# Reshape agar cocok dengan input model
mfcc_test_reshaped = np.expand_dims(mfcc_test_processed, axis=-1)

In [136]:
# Lakukan prediksi
predictions = model.predict(np.array([mfcc_test_reshaped]))

# Ambil kelas dengan probabilitas tertinggi
predicted_class = np.argmax(predictions) + 1

# Ambil label dari kelas
predicted_label = tokenizer.index_word[predicted_class]
print("Predicted Label:", predicted_label)

1/1 [==============================] - 0s 27ms/step
Predicted Label: the


# Percobaan 1

In [ ]:
print(tf.__version__)

2.14.0


In [ ]:
!pip install tensorflow-io[tensorflow]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 49.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
 

In [ ]:
import tensorflow_io as tfio

In [ ]:
def read_and_convert_to_tensor(csv_path, audio_folder, num_samples=None):
    df = pd.read_csv(csv_path)

    if num_samples:
        df = df.iloc[:num_samples]

    data = []
    labels = []

    for index, row in df.iterrows():
        file_name = row['filename']
        label = row['text']
        file_path = os.path.join(audio_folder, file_name)

        if os.path.exists(file_path):
            audio_binary = tf.io.read_file(file_path)
            audio = tfio.audio.decode_mp3(audio_binary)
            data.append(audio.numpy())
            labels.append(label)
        else:
            print(f"File not found: {file_path}")

    return np.array(data), np.array(labels)

# Menggunakan kode yang telah Anda berikan
train_csv_path = '/content/common-voice/cv-valid-train.csv'
train_audio_folder = '/content/common-voice/cv-valid-train'
dev_csv_path = '/content/common-voice/cv-valid-dev.csv'
dev_audio_folder = '/content/common-voice/cv-valid-dev'

# Gunakan parameter num_samples untuk membatasi jumlah sampel yang diambil
num_train_to_load = 1000  # Ganti dengan jumlah sampel yang Anda inginkan
num_dev_to_load = 100
train_data, train_labels = read_and_convert_to_tensor(train_csv_path, train_audio_folder, num_samples=num_train_to_load)
dev_data, dev_labels = read_and_convert_to_tensor(dev_csv_path, dev_audio_folder, num_samples=num_dev_to_load)

print("Train Data Shape:", train_data.shape)
print("Train Labels Shape:", train_labels.shape)
print("Dev Data Shape:", dev_data.shape)
print("Dev Labels Shape:", dev_labels.shape)

<ipython-input-18-da9de2656d60>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data), np.array(labels)


Train Data Shape: (1000,)
Train Labels Shape: (1000,)
Test Data Shape: (100,)
Test Labels Shape: (100,)


<ipython-input-18-da9de2656d60>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data), np.array(labels)


In [ ]:
print(len(train_data))

1000


In [ ]:
print(len(train_labels[0]))

62


In [ ]:
# Menghitung jumlah kata-kata unik dalam seluruh label pelatihan
unique_words = set(word for label in train_labels for word in label.split())

# Mendapatkan jumlah kata-kata unik
num_classes = len(unique_words)

print("Number of Unique Words (num_classes):", num_classes)

Number of Unique Words (num_classes): 1921


## Model LSTM

In [ ]:
train_data = train_data.reshape(train_data.shape[0], -1)
dev_data = dev_data.reshape(dev_data.shape[0], -1)

# Membuat tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
dev_dataset = tf.data.Dataset.from_tensor_slices((dev_data, dev_labels))

# Menyesuaikan input_shape dengan bentuk data suara Anda
input_shape = (None, 64000)  # Sesuaikan dengan panjang suara Anda

# Membangun model RNN
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=input_shape))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan menggunakan tf.data.Dataset
batch_size = 32  # Sesuaikan dengan kebutuhan Anda
epochs = 10

model.fit(train_dataset.batch(batch_size), epochs=epochs, validation_data=val_dataset.batch(batch_size))

ValueError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

#Membagi training dan testing
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Membangun model RNN
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(None, 64000)))  # Sesuaikan input_shape dengan bentuk data suara Anda
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))  # Sesuaikan num_classes dengan jumlah kelas Anda

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

ValueError: ignored

In [ ]:
from tensorflow.keras import layers, models

# Membangun model speech-to-text
model = models.Sequential()

# Lapisan konvolusi untuk mengekstrak fitur dari sinyal suara
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(panjang_sinyal, jumlah_fitur, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# LSTM untuk pemrosesan sekuens sinyal suara
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.LSTM(128))

# Lapisan tersembunyi dan dropout (opsional)
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))

# Lapisan output untuk transkripsi teks
model.add(layers.Dense(jumlah_kelas, activation='softmax'))

# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model (pastikan untuk menyesuaikan data dan label sesuai)
model.fit(train_data, train_labels_one_hot, epochs=10, validation_data=(val_data, val_labels_one_hot))